In [ ]:
import openai
import json
import requests
import pprint

def get_stock_quote(symbol):
    """Get info from stock symbol provided"""

    url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_KEY}'
    r = requests.get(url)
    data = r.json()
    return str(data)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "Give me some information about AAPL stock price"}],
        functions=[
            {
                "name": "get_stock_quote",
                "description": "Get the info about a stock price",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The ticker symbol, e.g. AAPL",
                        },
                    },
                    "required": ["symbol"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Step 3, call the function
        function_response = get_stock_quote(
            symbol=function_args.get("symbol"),
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "Give me some information about AAPL stock price"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())


## You can describe functions to GPT, and have the model intelligently choose to output a JSON object containing arguments to call those functions

## The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

### 1. Call the model with the user query and a set of functions defined in the functions parameter.
### 2. The model can choose to call a function; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may generate invalid JSON or hallucinate parameters).
### 3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
### 4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

In [ ]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)
print(type(get_current_weather('boston')))

In [ ]:
print(type(get_current_weather('boston')))

In [ ]:
import openai
import json

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )
    
    return response

In [ ]:
message = run_conversation()

In [ ]:
message

In [ ]:
message = run_conversation()["choices"][0]["message"]

In [ ]:
message

In [ ]:
# Does model wants to call a function????
if message.get("function_call"):
    function_name = message["function_call"]["name"]
    function_args = json.loads(message["function_call"]["arguments"])

    # here we actually call the function
    function_response = get_current_weather(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )

    # Step 4, send model the info on the function call and function response
    final_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": "What is the weather like in boston?"},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )
    print(final_response)